In [14]:
import requests
import os

In [15]:
STR_PROMPT = '''
You are an expert Python developer experienced with the Manim package. You should only give the code delimited by ```, do not give any descriptions or how to run it. Give me Python code that creates a Manim animation with the specifications below:
title: "Normal Distribution Visualization"
animation: Begin with a title slide with the title, then animate a normal distribution curve with the mean at the center and the standard deviations marked off. Show a point moving along the curve to represent a value and its corresponding probability.
important note: The code for the animation should be very simple. The animation should have little text if at all. There should not be any use of svgs or external files.
'''

In [16]:
# Your API key and other parameters
API_KEY = os.getenv('FIREWORKS_API_KEY')
ACCOUNT_ID = os.getenv('FIREWORKS_ACCOUNT_ID')
MODEL_ID = 'df1e9b8deec0413995e9a1e6b9109b9a'

# The API endpoint
url = 'https://api.fireworks.ai/inference/v1/completions'

# Headers for the request
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json',
}

# Data to be sent in the request
data = {
    'model': f'accounts/{ACCOUNT_ID}/models/{MODEL_ID}',
    'prompt': STR_PROMPT,
    "temperature": 0,
    'max_tokens': 32000,
}

# Make the POST request
response = requests.post(url, headers=headers, json=data)

# Check if the request was successful
if response.status_code == 200:
    # Process the response here
    print(response.json())
else:
    # Handle errors here
    print(f'Error: {response.status_code}, {response.text}')

{'id': 'd21e1895-8a5a-4800-b790-65f97e9790a5', 'object': 'text_completion', 'created': 1711291691, 'model': 'accounts/bassimfaizal-e0d5cc/models/df1e9b8deec0413995e9a1e6b9109b9a', 'choices': [{'index': 0, 'text': '\nfrom manim import *\n\nclass NormalDistribution(Scene):\n    def construct(self):\n        title = Text("Normal Distribution Visualization").scale(0.9)\n        self.play(Write(title))\n        self.wait(1)\n        self.play(FadeOut(title))\n\n        # Parameters\n        mean = 0\n        std_dev = 1\n        curve_color = BLUE\n        point_color = RED\n\n        # Normal Distribution Curve\n        x_range = np.linspace(-5, 5, 1000)\n        y_values = 1 / (std_dev * np.sqrt(2 * np.pi)) * np.exp(-((x_range - mean) ** 2) / (2 * std_dev ** 2))\n        normal_distribution_graph = FunctionGraph(lambda x: y_values[int(x * 1000)], x_range=x_range, color=curve_color)\n\n        # Mean Line\n        mean_line = Line(LEFT * mean, RIGHT * mean, color=WHITE)\n\n        # Standa

In [9]:
import csv
import json

# Define the file paths
jsonl_file_path = 'output_processed.jsonl'
csv_file_path = 'instruction_response.csv'

# Open the JSON Lines file and read lines
with open(jsonl_file_path, 'r') as jsonl_file:
    # Open the CSV file for writing
    with open(csv_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        # Write the header row
        csv_writer.writerow(['instruction', 'response'])
        
        # Process each line in the JSONL file
        for line in jsonl_file:
            # Convert the JSON string into a Python dictionary
            data = json.loads(line)
            # Extract the instruction and response
            instruction = data.get('instruction', '')
            response = data.get('response', '')
            # Write the instruction and response to the CSV file
            csv_writer.writerow([instruction, response])

In [ ]:
# and with the following as simply light reference if you need (note the code here may be completely incorrect) '{code_input}',
# generate or correct

In [19]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os

def generate_manim_code(user_input):
    # Retrieve the API key from the environment variables
    api_key = os.getenv('MISTRAL_API_KEY')
    
    # Define the model to use
    model = "mistral-large-latest"
    
    # Initialize the Mistral client with the API key
    client = MistralClient(api_key=api_key)
    
    # Format the prompt as specified, using user_input and code_input
    prompt = f"You are an expert python developer experienced in using Manim. Given the user query '{user_input}' generate the Manim code that fulfills the user's request. Please ensure there is no title slide used. Do not use MathTex. Your response should start with \"from manim import *\" and contain only the code."
    
    # Construct the messages list with a single ChatMessage containing the prompt
    messages = [
        ChatMessage(role="user", content=prompt)
    ]
    
    # Make the request to the chat API with no streaming
    chat_response = client.chat(
        model=model,
        messages=messages,
        temperature=0.2
    )
    
    # Access and return the content of the first message in the response
    # Check if there are choices and messages before accessing the content to avoid errors
    if chat_response.choices and chat_response.choices[0].message:
        return chat_response.choices[0].message.content
    else:
        return "No completion found or error in the response."


In [20]:
CODE_INPUT = '''

from manim import *\n\nclass NormalDistribution(Scene):\n    def construct(self):\n        title = Text("Normal Distribution Visualization").scale(0.9)\n        self.play(Write(title))\n        self.wait(1)\n        self.play(FadeOut(title))\n\n        # Parameters\n        mean = 0\n        std_dev = 1\n        curve_color = BLUE\n        point_color = RED\n\n        # Normal Distribution Curve\n        x_range = np.linspace(-5, 5, 1000)\n        y_values = 1 / (std_dev * np.sqrt(2 * np.pi)) * np.exp(-((x_range - mean) ** 2) / (2 * std_dev ** 2))\n        normal_distribution_graph = FunctionGraph(lambda x: y_values[int(x * 1000)], x_range=x_range, color=curve_color)\n\n        # Mean Line\n        mean_line = Line(LEFT * mean, RIGHT * mean, color=WHITE)\n\n        # Standard Deviation Lines\n        std_dev_lines = VGroup()\n        for _ in range(2):\n            std_dev_line = Line(UP * std_dev, DOWN * std_dev, color=WHITE)\n            std_dev_lines.add(std_dev_line)\n\n        # Point moving along the curve\n        point = Dot(radius=0.1, color=point_color)\n        point_path = always_redraw(lambda: FunctionGraph(lambda x: y_values[int(x * 1000)], x_range=[x_range[0], x_range[-1]], color=WHITE).point_from_proportion(np.random.normal()))\n        point.move_to(point_path.points[0])\n\n        # Animation\n        self.play(Create(normal_distribution_graph))\n        self.play(GrowFromCenter(mean_line))\n        self.play(FadeIn(std_dev_lines))\n        self.play(point.animate.shift(point_path.points[1] - point_path.points[0]))\n        self.play(point.animate.shift(point_path.points[2] - point_path.points[1]))\n        self.play(point.animate.shift(point_path.points[3] - point_path.points[2]))\n        self.play(point.animate.shift(point_path.points[4] - point_path.points[3]))\n        self.play(point.animate.shift(point_path.points[5] - point_path.points[4]))\n        self.play(point.animate.shift(point_path.points[6] - point_path.points[5]))\n        self.play(point.animate.shift(point_path.points[7] - point_path.points[6]))\n        self.play(point.animate.shift(point_path.points[8] - point_path.points[7]))\n        self.play(point.animate.shift(point_path.points[9] - point_path.points[8]))\n        self.wait(2)
'''

In [21]:
generate_manim_code(STR_PROMPT, 
                    CODE_INPUT)

'```python\nfrom manim import *\n\nclass NormalDistributionVisualization(Scene):\n    def construct(self):\n        axes = Axes(\n            x_range=[-4, 4, 1],\n            y_range=[0, 1, 0.25],\n            axis_color=BLUE,\n            x_axis_config={"numbers_to_include": [-3,0,3]},\n            tips=False,\n        )\n\n        labels = axes.get_axis_labels()\n\n        graph = axes.get_graph(lambda x : 0.4*np.exp(-x**2), color=YELLOW)\n        dot = Dot(color=RED).move_to(graph.points[0])\n\n        mean_line = DashedLine(start=axes.c2p(0,0), end=axes.c2p(0,0.4), color=GREEN)\n        std_dev_lines = [\n            DashedLine(start=axes.c2p(-1,0), end=axes.c2p(-1,0.4), color=GREEN),\n            DashedLine(start=axes.c2p(1,0), end=axes.c2p(1,0.4), color=GREEN)\n        ]\n\n        self.play(Create(axes), Write(labels))\n        self.wait(1)\n\n        self.play(Create(graph), Create(mean_line), Create(*std_dev_lines))\n        self.wait(1)\n\n        self.play(\n            dot.

In [ ]:
import csv
import sys
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

def get_prompt(idea):
    prompt=f"""
    You are an expert prompt engineer and python developer, experienced with the Manim package. Your task is to rewrite a user prompt in the given format.

    Here are some examples:
    1. User prompt: "explain gradient descent"
    Rewritten prompt: 
    You are an expert Python developer experienced with the Manim package.You should only give the code delimited by ```, do not give any descriptions or how to run it. Give me Python code that creates a Manim animation with the specifications below:
    title: "Gradient Descent Simplified"
    animation: Start with a title slide with the title, and then animate a conceptual representation of gradient descent by showing a ball rolling down a slope towards the lowest point, indicating the optimization process.
    important note: The code for the animation should be very simple. The animation should have little text if at all. There should not be any use of svgs or external files. 

    2. User prompt: "Explain Newton's First Law"
    Rewritten prompt: 
    You are an expert Python developer experienced with the Manim package.You should only give the code delimited by ```, do not give any descriptions or how to run it. Give me Python code that creates a Manim animation with the specifications below:
    title: "Newton's First Law in Motion"
    animation: Begin with a title slide with the title, then animate a stationary object remaining at rest, and a moving object continuing its motion at a constant velocity in a straight line, unless acted upon by an external force.
    important note: The code for the animation should be very simple. The animation should have little text if at all. There should not be any use of svgs or external files. 

    3. User prompt: "Demonstrate how a lever works"
    Rewritten prompt: 
    You are an expert Python developer experienced with the Manim package.You should only give the code delimited by ```, do not give any descriptions or how to run it. Give me Python code that creates a Manim animation with the specifications below:
    title: "The Principle of the Lever"
    animation: Start with a title slide with the title, and then animate a simple lever in action, showing how a small force applied at one end can lift a heavy weight at the other end, illustrating the concept of mechanical advantage.
    important note: The code for the animation should be very simple. The animation should have little text if at all. There should not be any use of svgs or external files. 

    Now rewrite the foll owing prompt:
    User prompt: "{idea}"
    Rewritten prompt: 
    """
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=0)
    return response.choices[0].message.content.strip()

def get_code(refined_prompt):
    messages = [{"role": "user", "content": refined_prompt}]
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=messages,
        temperature=0)
    return response.choices[0].message.content.strip()

# prompt= get_prompt('explain gradient descent')
# print(get_code(prompt))
if __name__ == "__main__":
    try:
        if len(sys.argv) < 2:
            raise ValueError("Please provide a CSV file.")
        input_csv_file = sys.argv[1]

        # Read the input CSV file
        with open(input_csv_file, 'r') as csvfile:
            reader = csv.DictReader(csvfile)
            rows = [row for row in reader]

        # Add the new columns
        for row in rows:
            idea = row['idea']
            print(f"Idea:{idea}")
            refined_prompt = get_prompt(idea)
            code = get_code(refined_prompt)
            row['prompt'] = refined_prompt
            row['code'] = code

        # Write the output to a new CSV file
        output_csv_file = 'output.csv'
        with open(output_csv_file, 'w') as csvfile:
            fieldnames = reader.fieldnames + ['prompt', 'code']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)

    except Exception as e:
        print("An error occurred:", e)
        sys.exit(1)

In [4]:
import json

# Path to the JSONL file
jsonl_file_path = 'output.jsonl'
new_jsonl_content = []

with open(jsonl_file_path, 'r') as file:
    for line in file:
        # Parse the JSON object in each line
        data = json.loads(line)
        # Extract code from the 'response' field, between the triple backticks
        code = data['response'].split('```')[1].strip()
        # Remove the leading "python\n" if present
        if code.startswith("python\n"):
            code = code.replace("python\n", "", 1).strip()
        # Update the 'response' field with the modified Python code
        data['response'] = code
        new_jsonl_content.append(data)

# Write the modified content back to the file
with open('output_processed.jsonl', 'w') as file:
    for item in new_jsonl_content:
        # Convert the Python dictionary to a JSON string and write it to the file
        file.write(json.dumps(item) + '\n')

